### Описание 
Компания купила сервис чтения книг по подписке. 


Цели и задачи:проанализировать базу данных, в которой информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Загрузка библиотек и подключение и получение доступа к базе данных

In [4]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [8]:
query = '''SELECT * FROM books LIMIT 5'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [9]:
query = '''SELECT * 
FROM publishers 
LIMIT 5;'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [17]:
query = '''SELECT * 
FROM authors
LIMIT 5;'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [18]:
query = '''SELECT * 
FROM ratings 
LIMIT 5; '''
pd.io.sql.read_sql(sql=text(query), con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [19]:
query = '''SELECT * 
FROM reviews 
LIMIT 5; '''
pd.io.sql.read_sql(sql=text(query), con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Вывод: Загрузили данные и познакимлись со всеми таблицами. Приступаем к решению задач. 

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [21]:
query = '''SELECT COUNT(*)
FROM books WHERE publication_date > '2000-01-01';'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,count
0,819


Вывод: после 1 января 2000 года выпустилось 819 книг. 

### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [38]:
query = ''' SELECT books.title,
    COUNT(DISTINCT reviews.review_id) AS review_count,
    AVG(ratings.rating) AS avg_rating
    FROM books
    LEFT JOIN reviews ON books.book_id = reviews.book_id
    LEFT JOIN ratings ON books.book_id = ratings.book_id
    GROUP BY books.book_id
    ORDER BY review_count DESC, avg_rating DESC'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,title,review_count,avg_rating
0,Twilight (Twilight #1),7,3.662500
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
2,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
3,The Book Thief,6,4.264151
4,The Glass Castle,6,4.206897
...,...,...,...
995,Disney's Beauty and the Beast (A Little Golden...,0,4.000000
996,Leonardo's Notebooks,0,4.000000
997,Essential Tales and Poems,0,4.000000
998,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


Вывод: книги с самым высоким пользовательским рейтингом не всегда совпадаются с книгами с самым большим количеством отзывов. 
    Самый высокий рейтинг - 5, самое большое количество отзывов - 7. Книги с самым большим количеством отзывов выглядят наиболее популярными, среди которых "Сумерки" и "Гарри Поттер". 

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры; 

In [39]:
query = '''SELECT    publishers.publisher,
    COUNT(books.book_id) AS book_count
    FROM books
    JOIN publishers ON books.publisher_id = publishers.publisher_id
    WHERE    books.num_pages > 50
    GROUP BY    publishers.publisher
    ORDER BY    book_count DESC
LIMIT 1; '''
pd.io.sql.read_sql(sql=text(query), con = engine)

,publisher,book_count
0,Penguin Books,42


Вывод: издательство, которое выпустило наибольшее количество книг, - Penguin Books. Всего - 42 книги. 

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [32]:
query = '''WITH book_rating_count AS (    
    SELECT book_id,
    COUNT(rating_id) as rating_count
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(rating_id) >= 50), author_avg_rating AS (
        SELECT authors.author,
        AVG(ratings.rating) as avg_rating
        FROM books  
        JOIN authors ON books.author_id = authors.author_id
        JOIN ratings ON books.book_id = ratings.book_id
        WHERE books.book_id IN 
        (SELECT book_id FROM book_rating_count)
        GROUP BY authors.author)
    SELECT author, avg_rating
    FROM author_avg_rating
    ORDER BY avg_rating DESC
    LIMIT 1;'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


Вывод: Среди самых популярных авторов, у книг которых более 50 оценок, самый высокий средней оценкой книг - J.K. Rowling/Mary GrandPre.  

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок. 

In [34]:
query = '''WITH user_rating_count AS (
            SELECT username,
            COUNT(rating_id) as rating_count
            FROM ratings
            GROUP BY username
            HAVING COUNT(rating_id) > 50),
            user_review_count AS (
            SELECT username,
            COUNT(review_id) as review_count
            FROM reviews
            WHERE username IN (SELECT username FROM user_rating_count)
            GROUP BY username)
            SELECT AVG(review_count) as avg_review_count
            FROM user_review_count;'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,avg_review_count
0,24.333333


Вывод: Среднее количество обзоров для книг от пользователей, которые постаивли более 50 оценок, - 24,3. 